In [ ]:
# Copyright 2022-2023 Sony Semiconductor Solutions Corp. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get list of "**Edge Applications**" from "**Console for AITRIOS**"

This notebook explains the workflow for retrieving the list of "**Edge Applications**" from "**Console for AITRIOS**". <br>
<br>
Instructions are described in [README.md](./README.md).

## Imports

In [ ]:
import pandas as pd
from console_access_library.client import Client
from IPython.display import display

## Load an instance of "**Console Access Library**" Client

Run this cell to restore instance variable 'client_obj' created on a notebook of Set up "**Console Access Library**".

In [ ]:
%store -r
client_obj: Client = client_obj

## Get the list of "**Edge Applications**"

In [ ]:
# Get an instance of Deployment API
deployment_obj = client_obj.get_deployment()

# Status of Vision and Sensing Application on Console
status_dictionary = {
    "0": "Importing completed (Before conversion)",
    "1": "Converting...",
    "2": "Conversion completed",
    "3": "Conversion failed",
}

response_names = []
response_versions = []
response_statuses = []

# Call an API to get Vision and Sensing Application list
try:
    response = deployment_obj.get_device_apps()
except Exception as e:
    # EXCEPTION
    raise e

if "result" in response and response["result"] != "SUCCESS":
    # ERROR
    raise ValueError("ERROR", response)

# SUCCESS
# Create output list
apps = response.get("apps", [])

for app in apps:
    versions = app.get("versions", [])
    for version in versions:
        response_names.append(app.get("name", ""))
        response_versions.append(version.get("version", ""))
        version_status = version.get("status", "")
        response_statuses.append(
            status_dictionary.get(
                version_status, "Unknown status '" + version_status + "'"
            )
        )

if len(response_names) == 0:
    raise Exception("The list of Vision and Sensing Applications is not found.")

output_frame = pd.DataFrame(
    {
        "application name": response_names,
        "version number": response_versions,
        "status": response_statuses,
    }
)
# setting backup
backup_max_rows = pd.options.display.max_rows
# output limit clear
pd.set_option("display.max_rows", None)
# View results
display(output_frame)
# setting restore
pd.set_option("display.max_rows", backup_max_rows)

In [ ]:
# Uncomment below to filter by app_name
# QUERY_MODEL_ID = "your_app_name"
# output_frame.query(f"`application name`=='{QUERY_MODEL_ID}'")